Install the required libraries if you have not already done so, then restart the kernel

In [ ]:
%pip install biopython==1.81 pandas numpy tqdm ipywidgets

Import libraries and necessary encoding information

In [ ]:
#Import libraries
from Bio import SeqIO, pairwise2
from Bio.Seq import Seq
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import pandas as pd
import numpy as np
import timeit
import time

#Import utility functions
from utils import toolkit as tk

#Import the needed classes
from classes.Log import Logging
from classes.Codes import Codes

# Create a class instance using the encoding info located in /library
main=Codes(n_cycles=2)

# Begin logging operations
log_records=Logging()
user_inputs={}

Specify Inputs

In [ ]:
##### INPUTS #####
# (Change as needed according according to the library's encoding scheme)
# Note that under the current encoding scheme, the PCR1 code is the first 6 bp so searching for it is unnecessary.

# Provide a short name for organizing the results in the output folder
user_inputs['NAME']= 'TestBatch1'

#####################
### INDEX INPUTS  ###
#####################

#This should be the number of BP that precedes the first bb encoding region (BB1)
user_inputs['BB1_START_IDX'] = 30

#This should be the number of BP that precedes the second bb encoding region (BB2)
user_inputs['BB2_START_IDX'] = 51

#This should be the number of BP that precedes the second bb encoding region (BB3)
user_inputs['BB3_START_IDX'] = None

#This should be the number of BP that precedes the second pcr encoding region (PCR2)
user_inputs['PCR2_START_IDX'] = 75


##############################
### OTHER REQUIRED INPUTS  ###
##############################

#The illumina adapter sequence used to preprocess the fastq file.
#Only reads containing this sequence will be extracted for analysis.
user_inputs['ADAPT_SEQ']='GATCGGAAGAGCACACGTCTG'

#Specify the length of the codes used for BB encoding
user_inputs['BB_ENCODING_LEN'] = 7

#Specify the length of the codes used for PCR encoding
user_inputs['PCR_ENCODING_LEN'] = 6

#Specify the max number of mismatch errors to allow in the encoding sequences
user_inputs['ENCODING_TOLERANCE'] = 1

# Define the path to your raw FASTQ file (This is the name of the file that will be submitted for preprocessing)
user_inputs['RAW_FASTQ_FILE'] = "ExampleData.fastq"

# Define the name you wish to use for saving the preprocessed FASTQ file (This will be the name of the preprocessing output file)
user_inputs['PROC_FASTQ_FILE'] = "Preprocessed_ExampleData.txt"



########################
### OPTIONAL INPUTS  ###
########################

#Set to "True" to test only the first N_TEST_SEQUENCES
#Set to 'False" to process the entire sequencing file
user_inputs['TEST_RUN']=False
user_inputs['N_TEST_SEQUENCES']=200000


main.load_user_inputs(user_inputs)
log_records.user_inputs=user_inputs

1) Process the raw fastq file <br>
 * This will find and extract only codes containing the query sequence and add them to the output file.

In [ ]:
'''
The preprocessing step performed here only identifies and aggregates sequences containing the illumina adapter sequences.
Trimming the adapter sequences offers no significant benefit and running a program or online tool using "cutadapt"
(as done previously) is not necessary.
'''


#Preprocess the fastq file and save a .txt file to the outputs folder
start_time = time.time()
n_records, w_records=main.preprocess_fastq()

#Store logging information for preprocessing
log_records.store_preprocessing_records(n_records, w_records, round(time.time()-start_time, ndigits=2))

#Print the results
print(f"Wrote {w_records} sequences to outputs/{user_inputs['NAME']}/{user_inputs['PROC_FASTQ_FILE']}")
print(f"{n_records-w_records} sequences were removed.")

**2) Extract the BB encoding sequences from the processed fastq file**


 * This will find and extract sequences at indices that theoretically correspond to the encoding sequences
 - Any sequencing or pcr errors are not accounted for.

In [ ]:
t0=timeit.default_timer()
main.extract_codes()
log_records.store_extraction_records(t0, timeit.default_timer())

#Preview the code_df
main.code_df.head()

3) Search for and correct encoding errors

In [ ]:
tqdm_notebook.pandas(desc="Correcting codes")
code_names=list(main.code_dict.keys())
correction_dict={}
rows_before=main.code_df.shape[0]

#For each code, update the code_df with additional columns for codes with correctable/uncorrectable errors
times=[]
for code in code_names:
    start_time = time.time()
    code_df=tk.update_code_df(main.code_df, code, main.code_dict[code])
    times.append(round(time.time()-start_time, ndigits=2))

#Create a copy of the code_df for filtering uncorrectable errors
corrected_df = code_df.copy()

#Create a corrected df containing only codes from the screen with less than or equal to the specified max_errors
#The corrected df is modified in place, iteratively filtering out rows for each code outside of tolerance
for idx, code in enumerate(code_names):
    corrected_df, correction_dict=tk.filter_code_df(code_df, corrected_df, code, correction_dict)


time_dict=dict(zip(code_names, times))
rows_after=corrected_df.shape[0]
log_records.store_correction_records(rows_before, rows_after, correction_dict, time_dict)

4) Count and organize the sequence data into a final dataframe

In [ ]:
#create a new df with the codes that will be used for counting
counted_df=pd.DataFrame()
counted_df['bb1']=corrected_df['corrected_bb1']
counted_df['bb2']=corrected_df['corrected_bb2']
counted_df['pcr1']=corrected_df['corrected_pcr1']
counted_df['pcr2']=corrected_df['corrected_pcr2']

#group the counted df in place
counted_df = counted_df.groupby(['bb1', 'bb2', 'pcr1', 'pcr2']).size().reset_index(name='Count')

#translate the pcr and bb codes and add them into the counted df
counted_df['pcr1_id']=counted_df['pcr1'].apply(lambda pcr1: main.pcr1_code_dict[pcr1])
counted_df['pcr2_id']=counted_df['pcr2'].apply(lambda pcr2: main.pcr2_code_dict[pcr2])
counted_df['bb1_id']=counted_df['bb1'].apply(lambda bb1: main.bb1_code_dict[bb1])
counted_df['bb1_Smiles']=counted_df['bb1_id'].apply(lambda bb1: main.bb1_smiles_dict[bb1])
counted_df['bb2_id']=counted_df['bb2'].apply(lambda bb2: main.bb2_code_dict[bb2])
counted_df['bb2_Smiles']=counted_df['bb2_id'].apply(lambda bb2: main.bb2_smiles_dict[bb2])

#pivot the counted_df (use pcr codes for the columns and index the rows according to BBs/Smiles)
final_df = counted_df.pivot(index=['bb1_id', 'bb1_Smiles','bb2_id', 'bb2_Smiles'], columns=['pcr1_id','pcr2_id'], values='Count').reset_index()

#fill the nan values with 0
final_df=final_df.fillna(0)

5. Output the results in two csv files:
    * 1) As absolute sequence counts
    * 2) As normalized sequence counts


In [ ]:
# Merge column names
merged_columns = [''.join(col).strip() for col in final_df.columns.values]
merged_columns = [str(col).replace('1a-','') for col in merged_columns]
merged_columns = [str(col).replace('1b-','') for col in merged_columns]

# Assign merged column names to DataFrame
final_df.columns = merged_columns

# Output the final DataFrame to csv with absolute sequence counts
final_df.to_csv(f'{main.output_location}/Results_AbsSeqCounts.csv')

#Convert to normalized sequence counts
#pcr_list=list(pcr_primer_1.values())
for pcr in final_df.columns[4:]:
    sum_value = final_df[pcr].sum()
    final_df[pcr]=final_df[pcr].apply(lambda count: (count/sum_value)*(len(final_df)))

# Output the final DataFrame to csv with normalized sequence counts
final_df.to_csv(f'{main.output_location}/Results_NormSeqCounts.csv')

# Output a log record
log_records.generate_log(f'{main.output_location}/Log.txt')